In [ ]:
import keras
print(keras.__version__)

In [2]:
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist
import numpy as np
from keras.utils import np_utils
from keras.layers import Activation,Dense,Dropout
from keras.models import Sequential

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)
 
# 6. Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
 
print(X_train.shape, y_train.shape, X_test.shape,y_test.shape,Y_train.shape,Y_test.shape)

model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
                              # of the layer above. Here, with a "rectified linear unit",
                              # we clamp all values below 0 to 0.
                           
model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax')) # This special "softmax" activation among other things,
                                 # ensures the output is a valid probaility distribution, that is
                                 # that its values are all non-negative and sum to 1.
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X_train, Y_train,
          batch_size=128, nb_epoch=10, verbose=1,
          validation_data=(X_test, Y_test))

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")



Training matrix shape (60000, 784)
Testing matrix shape (10000, 784)
(60000, 784) (60000,) (10000, 784) (10000,) (60000, 10) (10000, 10)


/Users/dc/anaconda/envs/tf35/lib/python3.5/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s - loss: 0.2478 - val_loss: 0.1069
Epoch 2/10
60000/60000 [==============================] - 5s - loss: 0.0997 - val_loss: 0.0791
Epoch 3/10
60000/60000 [==============================] - 6s - loss: 0.0729 - val_loss: 0.0708
Epoch 4/10
60000/60000 [==============================] - 6s - loss: 0.0552 - val_loss: 0.0669
Epoch 5/10
60000/60000 [==============================] - 6s - loss: 0.0469 - val_loss: 0.0657
Epoch 6/10
60000/60000 [==============================] - 6s - loss: 0.0369 - val_loss: 0.0690
Epoch 7/10
60000/60000 [==============================] - 6s - loss: 0.0339 - val_loss: 0.0723
Epoch 8/10
60000/60000 [==============================] - 6s - loss: 0.0328 - val_loss: 0.0703
Epoch 9/10
60000/60000 [==============================] - 6s - loss: 0.0262 - val_loss: 0.0705
Epoch 10/10
60000/60000 [==============================] - 6s - loss: 0.0262 - val_loss: 0.0707

In [3]:
from keras.models import model_from_json


json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
acc: 99.64%


In [4]:
#CNN KERAS
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist
import numpy as np
from keras.utils import np_utils
from keras.layers import Activation,Dense,Dropout,Flatten
from keras.optimizers import SGD
from keras.models import Sequential

num_class = 10  # number of class

input_shape = (1, 28, 28)
def preprocess_input(x):
    return x.reshape((-1, ) + input_shape) / 255.


def preprocess_output(y):
    return np_utils.to_categorical(y)

(train_x, train_y), (validation_x, validation_y) = mnist.load_data()
train_x, validation_x = map(preprocess_input, [train_x, validation_x])
train_y, validation_y = map(preprocess_output, [train_y, validation_y])
print('Loading MNIST data...')
print('train_x shape:', train_x.shape, 'train_y shape:', train_y.shape)
print('validation_x shape:', validation_x.shape,
      'validation_y shape', validation_y.shape)

print("Training matrix shape", train_x.shape,train_y.shape, validation_x.shape, validation_y.shape)

Y_train = np_utils.to_categorical(train_y, 10)
Y_test = np_utils.to_categorical(validation_y, 10)
 
print(train_x.shape, train_y.shape,validation_x.shape,validation_y.shape,Y_train.shape, Y_test.shape)

model = Sequential()

model.add(Conv2D(32,3, input_shape= (1,28,28),padding='same', name='conv1'))
model.add(MaxPooling2D(2, name='pool1', data_format="channels_first"))
model.add(Conv2D(64, 3, padding='same', name='conv2'))
model.add(MaxPooling2D(2, name='pool2', data_format="channels_first"))
model.add(Flatten(name='flatten'))
model.add(Dense(64, activation='relu', name='fc1'))
model.add(Dense(num_class, activation='softmax', name='fc2'))
model.compile(loss='categorical_crossentropy',
            optimizer=SGD(lr=0.01, momentum=0.9),
            metrics=['accuracy'])

model.fit(train_x, train_y,
          batch_size=128, nb_epoch=10,verbose=1,
          validation_data=(validation_x, validation_y))

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")



Loading MNIST data...
train_x shape: (60000, 1, 28, 28) train_y shape: (60000, 10)
validation_x shape: (10000, 1, 28, 28) validation_y shape (10000, 10)
Training matrix shape (60000, 1, 28, 28) (60000, 10) (10000, 1, 28, 28) (10000, 10)
(60000, 1, 28, 28) (60000, 10) (10000, 1, 28, 28) (10000, 10) (600000, 10) (100000, 10)


/Users/dc/anaconda/envs/tf35/lib/python3.5/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 10s - loss: 0.6538 - acc: 0.8034 - val_loss: 0.2173 - val_acc: 0.9343
Epoch 2/10
60000/60000 [==============================] - 10s - loss: 0.1638 - acc: 0.9514 - val_loss: 0.1148 - val_acc: 0.9654
Epoch 3/10
60000/60000 [==============================] - 10s - loss: 0.1096 - acc: 0.9666 - val_loss: 0.0835 - val_acc: 0.9739
Epoch 4/10
60000/60000 [==============================] - 10s - loss: 0.0874 - acc: 0.9732 - val_loss: 0.0690 - val_acc: 0.9782
Epoch 5/10
60000/60000 [==============================] - 10s - loss: 0.0734 - acc: 0.9777 - val_loss: 0.0590 - val_acc: 0.9802
Epoch 6/10
60000/60000 [==============================] - 11s - loss: 0.0639 - acc: 0.9802 - val_loss: 0.0541 - val_acc: 0.9820
Epoch 7/10
60000/60000 [==============================] - 10s - loss: 0.0579 - acc: 0.9818 - val_loss: 0.0508 - val_acc: 0.9833
Epoch 8/10
60000/60000 [==============================

In [ ]:
#keras CNN cifar


